# **Problem 2**

 **Installing Required dependencies**

In [1]:
!pip install transformers datasets tokenizers seqeval -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00


In [2]:
! pip install  accelerate -U
! pip install  transformers -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.5 MB/s eta 0:00:00


 **Importing required libraries**

In [49]:
import datasets
import numpy as np
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy

In [50]:
from transformers import pipeline

In [51]:
import accelerate
import transformers
import json

Defining label2id for manual_output

In [52]:
label2id_manual_output={'O': '0',
 'B-PER': '1',
 'I-PER': '2',
 'B-ORG': '3',
 'I-ORG': '4',
 'B-LOC': '5',
 'I-LOC': '6',
 'B-MISC': '7',
 'I-MISC': '8'}

Defining label2id for chatgpt_output

In [53]:
label2id_chatgpt_output = {
    'O': '0',
    'B-PER': '1',
    'I-PER': '2',
    'B-ORG': '3',
    'I-ORG': '4',
    'B-LOC': '5',
    'I-LOC': '6',
    'B-GPE': '7',
    'I-GPE': '8',
    'B-DATE': '9',
    'I-DATE': '10',
    'B-TIME': '11',
    'I-TIME': '12',
    'B-MONEY': '13',
    'I-MONEY': '14',
    'B-PERCENT': '15',
    'I-PERCENT': '16',
    'B-FAC': '17',
    'I-FAC': '18',
    'B-PROD': '19',
    'I-PROD': '20',
    'B-EVENT': '21',
    'I-EVENT': '22',
    'B-WORK_OF_ART': '23',
    'I-WORK_OF_ART': '24',
    'B-LAW': '25',
    'I-LAW': '26',
    'B-LANGUAGE': '27',
    'I-LANGUAGE': '28'
}


In [54]:
#processing manualoutputs and chatgpt outputs to convert ners to nertags
def process_file(file_path, label2id):
    tokens=[]
    ner_tags=[]
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            json_obj = json.loads(line)
            tokens.append(json_obj['words'])
            ners=[]
            for i in json_obj['ner']:
                ners.append(label2id[i])
            ner_tags.append(ners)

    directory={}
    directory['tokens']=tokens
    directory['ner_tags']=ner_tags
    return directory

In [55]:
def tokenize_and_align_labels_for_our_outputs(examples,tokenizer,label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    dictioray_of_input_idsandlabels={}
    dictioray_of_input_idsandlabels['input_ids']=tokenized_inputs["input_ids"]
    dictioray_of_input_idsandlabels['labels']=labels
    return dictioray_of_input_idsandlabels

In [56]:
def decode_tokenized_datasets(tokenized_datasets, tokenizer):
    manual_outputs=[]
    for item in range(len(tokenized_datasets['input_ids'])):
        temp=[]
        for tokenid in range(len(tokenized_datasets['input_ids'][item])):
            if(tokenized_datasets['input_ids'][item][tokenid]==101 or tokenized_datasets['input_ids'][item][tokenid]==102 ):
                continue
            else:
                pair=[]
                pair.append(tokenizer.decode(tokenized_datasets['input_ids'][item][tokenid]))
                pair.append(tokenized_datasets['labels'][item][tokenid])
            temp.append(pair)
        manual_outputs.append(temp[1:-1])
    return manual_outputs

In [57]:
def convert_outputs_to_desired_format(file_path,label2id,tokenizer):
  result=result = process_file(file_path, label2id)
  tokenized_datasets=tokenize_and_align_labels_for_our_outputs(result, tokenizer)
  outputs_in_wanted_format=decode_tokenized_datasets(tokenized_datasets, tokenizer)
  return outputs_in_wanted_format


In [58]:
def tokenize_and_align_labels(examples,label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [59]:
from sklearn.metrics import f1_score
def compute_metrics(eval_preds):
    pred_logits, labels = eval_preds

    pred_logits = np.argmax(pred_logits, axis=2)
    predictions = [
        [label_list[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(pred_logits, labels)
    ]

    true_labels = [
      [label_list[l] for (eval_preds, l) in zip(prediction, label) if l != -100]
       for prediction, label in zip(pred_logits, labels)
   ]
    results = metric.compute(predictions=predictions, references=true_labels)
    return {
   "precision": results["overall_precision"],
   "recall": results["overall_recall"],
   "macro_f1": results["overall_f1"],
  "accuracy": results["overall_accuracy"],
  }

In [60]:
def process_ner_result(tokenizer, nlp, label2id, example):
    tokens = tokenizer.tokenize(example)
    ner_results = nlp(example)
    output_dict = []
    l = []
    for i in ner_results:
        pair = []
        pair.append(i['word'])
        pair.append(label2id[i['entity']])
        l.append(pair)

    i = 0
    j = 0
    while j < len(tokens) and i < len(l):
        if l[i][0] == tokens[j]:
            pair = []
            pair.append(l[i][0])
            pair.append(l[i][1])
            output_dict.append(pair)
            i += 1
        else:
            pair = []
            pair.append(tokens[j])
            pair.append(0)
            output_dict.append(pair)
        j += 1

    while j < len(tokens):
        pair = []
        pair.append(tokens[j])
        pair.append(0)
        output_dict.append(pair)
        j += 1

    return output_dict[:-1]

In [61]:
def getting_outputs_from_our_model(file_path,tokenizer, nlp, label2id):
  results_from_our_model=[]
  with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
      results_from_our_model.append(process_ner_result(tokenizer, nlp, label2id, line))
  return results_from_our_model

In [62]:
def flatten_list(list_of_lists):
    flattened_list = [item for sublist in list_of_lists for item in sublist]
    outputlist=[item[1] for item in flattened_list]
    return outputlist

**Defining training arguments with 5 epochs**

In [63]:
bert_args = TrainingArguments(
    "test-bert",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy="no"
)

**Loading datasets and splitting them in to 70% training, 10% validation,and 20% test sets**

In [64]:
from datasets import load_dataset
lang='te'
raw_datasets = load_dataset('ai4bharat/naamapadam', lang)

num_samples = len(raw_datasets['train']) // 5
subset_dataset = raw_datasets['train'].shuffle(seed=42).select(range(num_samples))
train_size = int(0.7 * num_samples)
val_size = int(0.1 * num_samples)
test_size = num_samples - train_size - val_size

train_data = subset_dataset.select(range(train_size))
val_data = subset_dataset.select(range(train_size, train_size + val_size))
test_data = subset_dataset.select(range(train_size + val_size, num_samples))

# Now you have 70% training, 10% validation, and 20% test data
raw_datasets['train']=train_data
raw_datasets['validation']=val_data
raw_datasets['test']=test_data

In [65]:
label_list = raw_datasets["train"].features["ner_tags"].feature.names

In [66]:
from seqeval.metrics import classification_report
from sklearn.metrics import f1_score
metric = datasets.load_metric("seqeval")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


## **INDIC-BERT**

In [67]:
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")

In [68]:
tokenized_datasets = raw_datasets.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/71083 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/20311 [00:00<?, ? examples/s]

Map:   0%|          | 0/10154 [00:00<?, ? examples/s]

In [69]:
model = AutoModelForTokenClassification.from_pretrained("ai4bharat/indic-bert", num_labels=7)

Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [70]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [71]:
trainer = Trainer(
    model,
    bert_args,
   train_dataset=tokenized_datasets["train"],
   eval_dataset=tokenized_datasets["validation"],
   data_collator=data_collator,
   tokenizer=tokenizer,
   compute_metrics=compute_metrics,
)

In [72]:
trainer.train()

TrainOutput(global_step=22215, training_loss=0.32951485313812195, metrics={'train_runtime': 4451.0536, 'train_samples_per_second': 79.85, 'train_steps_per_second': 4.991, 'total_flos': 1088191818524298.0, 'train_loss': 0.32951485313812195, 'epoch': 5.0})

Test set macro f1 score in trained bert

In [73]:
predictions, labels, bert_metrics = trainer.predict(tokenized_datasets["test"])
print(bert_metrics['test_macro_f1'])

0.735874195424947


Train set macro f1 score in trained bert

In [74]:
predictions, labels, bert_metrics = trainer.predict(tokenized_datasets["train"])
print(bert_metrics['test_macro_f1'])

0.822118283943172


Validation set macro f1 score in trained bert

In [75]:
predictions, labels, bert_metrics = trainer.predict(tokenized_datasets["validation"])
print(bert_metrics['test_macro_f1'])

0.7338483491610491


In [76]:
model.save_pretrained("bert_model")
tokenizer.save_pretrained("bert_tokenizer")
bert_tokenizer=tokenizer

In [77]:
id2label = {
    str(i): label for i,label in enumerate(label_list)
}
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}

In [78]:
config = json.load(open("bert_model/config.json"))
config["id2label"] = id2label
config["label2id"] = label2id
json.dump(config, open("bert_model/config.json","w"))
bert_model_fine_tuned = AutoModelForTokenClassification.from_pretrained("bert_model")
nlp_bert = pipeline("ner", model=bert_model_fine_tuned, tokenizer= bert_tokenizer)

In [79]:
my_bert_model_outputs=getting_outputs_from_our_model('25sentences.txt',tokenizer, nlp_bert, label2id)

In [80]:
my_bert_model_outputs=flatten_list(my_bert_model_outputs)

In [81]:
my_manual_outputs_bert=convert_outputs_to_desired_format('manual_outputs.txt',label2id_manual_output,tokenizer)

In [82]:
my_manual_outputs_bert=flatten_list(my_manual_outputs_bert)

In [83]:
chatgpt_outputs_bert=convert_outputs_to_desired_format('chatgpt_outputs.txt',label2id_chatgpt_output,tokenizer)

In [84]:
chatgpt_outputs_bert=flatten_list(chatgpt_outputs_bert)

# **Problem 4**

In [85]:
from sklearn.metrics import precision_score, recall_score, f1_score

def calculate_and_print_metrics(ground_truth, predicted):
    precision = precision_score(ground_truth, predicted, average='macro')
    recall = recall_score(ground_truth, predicted, average='macro')
    f1 = f1_score(ground_truth, predicted, average='macro')

    print("Precision:", precision)
    print("Recall:", recall)
    print("Macro F1 Score:", f1)
    print("\n")


Question 2 vs Question1(BERT)

In [86]:
my_bert_model_outputs = [str(i) for i in my_bert_model_outputs]
calculate_and_print_metrics(my_manual_outputs_bert, my_bert_model_outputs)

Precision: 0.5368707624805187
Recall: 0.46696864994737336
Macro F1 Score: 0.48899025722555134




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Question 3 vs Question1(BERT)

In [87]:
calculate_and_print_metrics(my_manual_outputs_bert, chatgpt_outputs_bert)

Precision: 0.5300113597985938
Recall: 0.4575926652328974
Macro F1 Score: 0.48505593629977745




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Question 1 vs Question 3 without tokenizing

In [88]:
#processing manualoutputs and chatgpt outputs to convert ners to nertags
def process_file_without_tokenizing(file_path, label2id):
    tokens=[]
    ner_tags=[]
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            json_obj = json.loads(line)
            tokens.append(json_obj['words'])
            ner_tags.append(json_obj['ner'])
    directory={}
    directory['tokens']=tokens
    directory['ner_tags']=ner_tags
    return directory
manual_ops=process_file('manual_outputs.txt',label2id_manual_output)['ner_tags']
chatgpt_ops=process_file('chatgpt_outputs.txt',label2id_chatgpt_output )['ner_tags']
for i in range(24):
    if(len(manual_ops[i])!=len(chatgpt_ops[i])):
        print(i," ",len(manual_ops[i])," ",len(chatgpt_ops[i]))
        print(manual_ops[i])
        print(chatgpt_ops[i])
manual_ops = [item for sublist in manual_ops for item in sublist]
chatgpt_ops=[item for sublist in chatgpt_ops for item in sublist]
calculate_and_print_metrics(manual_ops, chatgpt_ops)


Precision: 0.5559871229617188
Recall: 0.46595379258422737
Macro F1 Score: 0.49863023416742425




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
